# Highways that are Transit Deserts

These highway corridors have a dearth of bus transit service. 

## Criteria
* 10-mile corridors 
* No transit routes are found within 2 miles of the highway.

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import branca
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

catalog = intake.open_catalog("./*.yml")

In [ ]:
gdf = catalog.highway_transit_deserts.read()

In [ ]:
def make_district_map(gdf: gpd.GeoDataFrame, district: int): 
    gdf2 = gdf[gdf.District==district].drop(
        columns = ["hwy_segment_id", "segment_sequence"])
    
    m = gdf2.explore("Route", 
                     categorical = True, 
                     tiles = "CartoDB Positron"
                    )
    
    display(m)

## District Maps

In [ ]:
districts = gdf.District.unique().tolist()

for i in sorted(districts):
    display(HTML(f"<h3>District {i}</h3>"))
    
    make_district_map(gdf, i)

<a id="top"></a>       